<h2>개인 구글 드라이브와 colab 연동</h2>

In [35]:
# from google.colab import drive
# drive.mount("/gdrive", force_remount=True)
file_path = "SMSSpamCollection"


<h2>"SMSSpamCollection" 데이터를 읽고 문장과 정답을 분리하여 각 리스트에 저장</h2>

<pre>
<b>1. 데이터의 형태(SMSSpamCollection)</b>
  라벨(스팸 또는 햄) \t(tab) 문장
  
  위와 같은 형태로 저장되어 있음
  
  예시)
    ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    spam\tCustomer service annoncement. You have a New Years delivery waiting for you. Please call 07046744435 now to arrange delivery
    ...
  
  따라서 입력 데이터를 읽고 \t을 기준으로 입력 문장을 분리한 후에 문장과 라벨을 각각 x_data, y_data 리스트에 저장

<b>2. x_data, y_data 형태</b>
  x_data = [ 문장1, 문장2, 문장3, ... ]
  y_data = [ 문장1의 라벨, 문장2의 라벨, 문장3의 라벨, ... ]
</pre>

In [36]:
import nltk

######
print(f"english stop word: {len(nltk.corpus.stopwords.words('english'))}")
print(nltk.corpus.stopwords.words('english')[:10])
stop_words = set(nltk.corpus.stopwords.words('english')) 
######
# 파일 읽기
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()

x_data, y_data = [], []
for line in lines:
  pieces = line.strip().split('\t')
  label, sentence = pieces[0], pieces[1]

  ######
  sentence = sentence.lower().split()
  sentence = ' '.join([word for word in sentence if word not in stop_words])
  ######

  x_data.append(sentence)
  y_data.append(label)

print("x_data의 개수 : {}".format(len(x_data)))
print("y_data의 개수 : {}".format(len(y_data)))

english stop word: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
x_data의 개수 : 1500
y_data의 개수 : 1500


In [37]:
import keras
print(keras.__version__)
import tensorflow
print(tensorflow.__version__)

2.15.0
2.15.0


<h2>Tokenizer 라이브러리를 사용하여 입력 문장을 index로 치환</h2>

<pre>
<b>1. tokenizer.fit_on_texts(data) 함수를 이용하여 각 단어를 index로 치환하기 위한 딕셔너리 생성</b>
   생성된 딕셔너리는 tokenizer 객체 안에 저장됨
  
  tokenizer.fit_on_texts(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return
    X
    
  딕셔너리 예시)
    {'to': 0, 'i': 1, 'you': 2, 'a': 3, 'the': 4, 'and': 5, 'for': 6 ... }
    
<b>2. tokenizer.texts_to_matrix(data) 함수를 이용하여 각 단어를 one-hot 벡터로 표현하며 문장은 각 단어의 one-hot 벡터의 합으로 표현</b>

  tokenizer.texts_to_sequences(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return :
    indexing 된 리스트

  예시)
    전체 단어 사전 : {'i': 0, 'am': 1, 'happy': 2, 'sad': 3}
    i     : [1, 0, 0, 0]
    am    : [0, 1, 0, 0]
    happy : [0, 0, 1, 0]
    sad   : [0, 0, 0, 1]
    
    문장 1 : i am happy   -> [1, 1, 1, 0]
    문장 2 : i am sad     -> [1, 1, 0, 1]
    문장 3 : i am sad sad -> [1, 1, 0, 2]
    
  indexing 예시)
    x_data indexing 하기 전 : Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    x_data indexing 하기 후 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    y_data indexing 하기 전 : ham
    y_data indexing 하기 후 : 1
</pre>

In [38]:
from keras.preprocessing.text import Tokenizer

# spam, ham 라벨을 대응하는 index로 치환하기위한 딕셔너리
label2index = {'spam':0, 'ham':1}
index2label = {0:"spam", 1:"ham"}

# indexing 한 데이터를 넣을 리스트 선언
indexing_x_data, indexing_y_data = [], []

for label in y_data:
  indexing_y_data.append(label2index[label])

tokenizer = Tokenizer(num_words=200) #dictionary 최대 개수 200개로 (말뭉치 크기보다 작으면 무시)

# x_data를 사용하여 딕셔너리 생성
tokenizer.fit_on_texts(x_data)

# x_data에 있는 각 문장들을 one-hot 벡터의 합으로 치환하고 그 결과값을 indexing_x_data에 저장
indexing_x_data = tokenizer.texts_to_matrix(x_data).tolist()

print("x_data indexing 하기 전 : {}".format(x_data[0]))
print("x_data indexing 하기 후 : {}".format(indexing_x_data[0]))
print("y_data indexing 하기 전 : {}".format(y_data[0]))
print("y_data indexing 하기 후 : {}".format(indexing_y_data[0]))

x_data indexing 하기 전 : go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...
x_data indexing 하기 후 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

<h2>SVM 모델 학습</h2>

<pre>    
<b>1. 입력 데이터를 9 대 1 비율로 나누어 학습, 평가에 사용</b>
  
<b>2. svm.fit(x, y) 함수를 사용하여 SVM 모델 학습</b>
  svm.fit(x, y)
  args
    x : indexing 된 문장들이 있는 리스트
    y : x의 각 문장에 대응하는 라벨이 있는 리스트
  return :
    X
</pre>

In [39]:
# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
number_of_train_data = int(len(indexing_x_data)*0.9)

train_x, train_y = indexing_x_data[:number_of_train_data], indexing_y_data[:number_of_train_data]
test_x, test_y = indexing_x_data[number_of_train_data:], indexing_y_data[number_of_train_data:]

print("train_x의 개수 : {}".format(len(train_x)))
print("train_y의 개수 : {}".format(len(train_y)))
print("test_x의 개수 : {}".format(len(test_x)))
print("test_y의 개수 : {}".format(len(test_y)))


train_x의 개수 : 1350
train_y의 개수 : 1350
test_x의 개수 : 150
test_y의 개수 : 150


In [40]:
from sklearn.svm import SVC
svm = SVC(kernel='linear',C=0.01)
svm.fit(train_x, train_y)

SVC(C=0.01, kernel='linear')

<h2>SVM 모델을 이용한 평가</h2>

<pre>
<b>1. svm.predict(data) 함수를 사용하여 SVM 모델을 이용하여 평가</b>
  
  svm.predict(data)
  args
    data : indexing 된 문장들이 있는 리스트
  return :
    입력 문장들에 대한 모델의 출력 라벨 리스트
    
<b>2. 성능 측정</b>
  정답 라벨과 모델의 출력 라벨을 비교하여 성능 측정
  
<b>3. tokenizer.sequences_to_texts(data) 함수를 이용하여 indexing 된 데이터를 단어로 치환</b>

  tokenizer.sequences_to_texts(data)
  args
    data : indexing 된 리스트
  return :
    단어로 치환된 리스트
    
  예시)
    [38, 93, 239, 240, 241, 242, 53, 11, 243, 72, 94, 244, 245, 126, 246, 247, 73, 74, 248, 127] -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    
<b>4. 입력 문장에 대한 모델의 출력과 정답 출력</b>

</pre>

In [41]:
from sklearn.metrics import accuracy_score

predict = svm.predict(test_x)
accuracy = accuracy_score(test_y, predict)

print("Accuracy: {}".format(accuracy))

for index in range(len(x_data[number_of_train_data:number_of_train_data+5])):
  print()
  print("문장 : {}".format(x_data[index]))
  print("정답 : {}".format(index2label[test_y[index]]))
  print("모델 출력 : {}".format(index2label[predict[index]]))

Accuracy: 0.9133333333333333

문장 : go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...
정답 : spam
모델 출력 : ham

문장 : ok lar... joking wif u oni...
정답 : ham
모델 출력 : ham

문장 : free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's
정답 : ham
모델 출력 : ham

문장 : u dun say early hor... u c already say...
정답 : ham
모델 출력 : ham

문장 : nah think goes usf, lives around though
정답 : ham
모델 출력 : ham


In [42]:
from sklearn.svm import SVC

kernel_functions = ['linear', 'poly', 'sigmoid', 'rbf']
for kernel_function in kernel_functions:
  print("#########################################################")
  print("current Kernel Function : {}".format(kernel_function))
  svm = SVC(kernel=kernel_function)
  svm.fit(train_x, train_y)
  predict = svm.predict(test_x)
  accuracy = accuracy_score(test_y, predict)

  print("Accuracy: {}\n".format(str(accuracy*100)[:5]))


#########################################################
current Kernel Function : linear
Accuracy: 94.0

#########################################################
current Kernel Function : poly
Accuracy: 94.66

#########################################################
current Kernel Function : sigmoid
Accuracy: 96.0

#########################################################
current Kernel Function : rbf
Accuracy: 96.0

